In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np
import requests
import mysql.connector

: 

In [ ]:
df = pd.read_csv('/mnt/d/Adalab/ejercicios-de-adalab/modulo-2/8-ETL-mod-2/files/datos_union_clima_ataques.csv', index_col = 0)
df.head(2)

: 

- CREACIÓN BBDD

In [ ]:
def crear_bbdd(nombre_bbdd):
    
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="Marinadalab" # aquí tendréis que poner vuestra contraseña de MySQL
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

: 

In [ ]:
mydb = crear_bbdd('tiburones')

: 

CREACIÓN TABLAS

In [ ]:
# primero crearnos una función que valga para crear o insertar datos de cq tabla, con 3 parametros: 
# Nombre BBDD           # Contraseña servidor de SQL    # Query a insertar
                                                                                            

def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

: 

In [ ]:
df.columns

: 

In [ ]:
df.info()

: 

In [ ]:
# Definimos las distintas *queries* que crearán nuestras tablas. 
# Tabla ataques

tabla_ataques = '''
CREATE TABLE IF NOT EXISTS 'tiburones'.'ataques' (
  'idataque' INT NOT NULL AUTO_INCREMENT,
  'YEAR' INT NOT NULL,
  'TYPE' VARCHAR (100) NOT NULL,
  'COUNTRY' VARCHAR (100) NOT NULL,
  'AREA' VARCHAR (100) NOT NULL,
  'LOCATION' VARCHAR (100) NOT NULL,
  'ACTIVITY' VARCHAR (100) NOT NULL,
  'NAME' VARCHAR (100) NOT NULL,
  'AGE' VARCHAR (100) NOT NULL,
  'TIME' VARCHAR (100) NOT NULL,
  'SPECIES_' VARCHAR (100) NOT NULL,
  'INJURY'  VARCHAR (100) NOT NULL,
  'FECHA_LIMPIA' VARCHAR (100) NOT NULL,
  'FATAL' VARCHAR (100) NOT NULL,
  'SEX' VARCHAR (100) NOT NULL,
  'PAÍS' VARCHAR (100) NOT NULL,
  PRIMARY KEY ('idataque'))
ENGINE = InnoDB;
'''




: 

In [ ]:
# Tabla clima

tabla_clima = """
CREATE TABLE IF NOT EXISTS 'tiburones'.'clima' (
  'idclima' INT NOT NULL AUTO_INCREMENT',
  'rh_950mb' DECIMAL NOT NULL,        
  'rh_900mb' DECIMAL NOT NULL,          
  'rh_850mb' DECIMAL NOT NULL,          
  'rh_800mb' DECIMAL NOT NULL,         
  'rh_750mb' DECIMAL NOT NULL,        
  'rh_700mb ' DECIMAL NOT NULL,         
  'rh_650mb' DECIMAL NOT NULL,         
  'rh_600mb' DECIMAL NOT NULL,          
  'rh_550mb' DECIMAL NOT NULL,       
  'rh_500mb' DECIMAL NOT NULL,         
  'rh_450mb' DECIMAL NOT NULL,          
  'rh_400mb' DECIMAL NOT NULL,          
  'rh_350mb ' DECIMAL NOT NULL,         
  'rh_300mb' DECIMAL NOT NULL,          
  'rh_250mb' DECIMAL NOT NULL,          
  'rh_200mb' DECIMAL NOT NULL,          
  'wind_950mb' DECIMAL NOT NULL,       
  'wind_900mb' DECIMAL NOT NULL,       
  'wind_850mb' DECIMAL NOT NULL,        
  'wind_800mb' DECIMAL NOT NULL,        
  'wind_750mb' DECIMAL NOT NULL,        
  'wind_700mb' DECIMAL NOT NULL,       
  'wind_650mb' DECIMAL NOT NULL,        
  'wind_600mb' DECIMAL NOT NULL,        
  'wind_550mb' DECIMAL NOT NULL,       
  'wind_500mb' DECIMAL NOT NULL,       
  'wind_450mb' DECIMAL NOT NULL,       
  'wind_400mb' DECIMAL NOT NULL,       
  'wind_350mb' DECIMAL NOT NULL,       
  'wind_300mb' DECIMAL NOT NULL,       
  'wind_250mb' DECIMAL NOT NULL,       
  'wind_200mb' DECIMAL NOT NULL,       
  'index' DECIMAL NOT NULL,            
  'timepoint' DECIMAL NOT NULL,         
  'cloudcover' DECIMAL NOT NULL,         
  'highcloud' DECIMAL NOT NULL,          
  'midcloud' DECIMAL NOT NULL,         
  'lowcloud' DECIMAL NOT NULL,          
  'temp2m' DECIMAL NOT NULL,            
  'lifted_index' DECIMAL NOT NULL,      
  'rh2m' DECIMAL NOT NULL,               
  'msl_pressure' DECIMAL NOT NULL,       
  'prec_amount' DECIMAL NOT NULL,       
  'snow_depth' DECIMAL NOT NULL,        
  'wind10m.direction' DECIMAL NOT NULL, 
  'wind10m.speed' DECIMAL NOT NULL,
  PRIMARY KEY ('idclima'))
ENGINE = InnoDB;"""
      

: 

In [ ]:
# Llamamos a la función para crear cada una de las tablas de nuestra BBDD.

crear_insertar_tabla('tiburones', 'Marinadalab', tabla_ataques)  


: 

In [ ]:
crear_insertar_tabla('tiburones', 'Marinadalab', tabla_clima)  


: 

- INSERCIÓN DATOS

In [ ]:

for indice, fila in df.iterrows(): 
        query_ataques = f"""
            INSERT INTO ataques ('YEAR','TYPE','COUNTRY','AREA','LOCATION','ACTIVITY','NAME','AGE','TIME',
            'SPECIES_' ,'INJURY' , 'FECHA_LIMPIA' , 'FATAL' , 'SEX','PAÍS' 
            VALUES ("{fila['YEAR']}", "{fila['TYPE']}","{fila['COUNTRY']}","{fila['AREA']}","{fila['LOCATION']}",
            "{fila['ACTIVITY']}","{fila['NAME']}","{fila['AGE']}","{fila['TIME']}","{fila['SPECIES_']}","{fila['INJURY']}",
            "{fila['FECHA_LIMPIA']}","{fila['FATAL']}","{fila['SEX']}","{fila['PAÍS']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
crear_insertar_tabla("tiburones", "Marinadalab", query_ataques)

: 

In [ ]:
for indice, fila in df.iterrows(): 
        query_clima = f"""
            INSERT INTO ataques ( 
    'rh_950mb'DECIMAL NOT NULL,
    'rh_900mb' DECIMAL NOT NULL,
    'rh_850mb' DECIMAL NOT NULL,         
  'rh_800mb' DECIMAL NOT NULL,         
  'rh_750mb' DECIMAL NOT NULL,        
  'rh_700mb ' DECIMAL NOT NULL,         
  'rh_650mb' DECIMAL NOT NULL,         
  'rh_600mb' DECIMAL NOT NULL,          
  'rh_550mb' DECIMAL NOT NULL,       
  'rh_500mb' DECIMAL NOT NULL,         
  'rh_450mb' DECIMAL NOT NULL,          
  'rh_400mb' DECIMAL NOT NULL,          
  'rh_350mb ' DECIMAL NOT NULL,         
  'rh_300mb' DECIMAL NOT NULL,          
  'rh_250mb' DECIMAL NOT NULL,          
  'rh_200mb' DECIMAL NOT NULL,          
  'wind_950mb' DECIMAL NOT NULL,       
  'wind_900mb' DECIMAL NOT NULL,       
  'wind_850mb' DECIMAL NOT NULL,        
  'wind_800mb' DECIMAL NOT NULL,        
  'wind_750mb' DECIMAL NOT NULL,        
  'wind_700mb' DECIMAL NOT NULL,       
  'wind_650mb' DECIMAL NOT NULL,        
  'wind_600mb' DECIMAL NOT NULL,        
  'wind_550mb' DECIMAL NOT NULL,       
  'wind_500mb' DECIMAL NOT NULL,       
  'wind_450mb' DECIMAL NOT NULL,       
  'wind_400mb' DECIMAL NOT NULL,       
  'wind_350mb' DECIMAL NOT NULL,       
  'wind_300mb' DECIMAL NOT NULL,       
  'wind_250mb' DECIMAL NOT NULL,       
  'wind_200mb' DECIMAL NOT NULL,       
  'index' DECIMAL NOT NULL,            
  'timepoint' DECIMAL NOT NULL,         
  'cloudcover' DECIMAL NOT NULL,         
  'highcloud' DECIMAL NOT NULL,          
  'midcloud' DECIMAL NOT NULL,         
  'lowcloud' DECIMAL NOT NULL,          
  'temp2m' DECIMAL NOT NULL,            
  'lifted_index' DECIMAL NOT NULL,      
  'rh2m' DECIMAL NOT NULL,               
  'msl_pressure' DECIMAL NOT NULL,       
  'prec_amount' DECIMAL NOT NULL,       
  'snow_depth' DECIMAL NOT NULL,        
  'wind10m.direction' DECIMAL NOT NULL, 
  'wind10m.speed' DECIMAL NOT NULL,
                
                
                
                
                
                
                
                
                
                
                
                
                
                
            VALUES ( "{fila["YEAR"]}", "{fila['TYPE']},"{fila['COUNTRY']},"{fila['AREA']},"{fila['LOCATION']},
            "{fila['ACTIVITY']},"{fila['NAME']},"{fila['AGE']},"{fila['TIME']},"{fila['SPECIES_']},"{fila['INJUTY']},
            "{fila['FECHA_LIMPIA']},"{fila['FATAL']}","{fila['SEX']}","{fila['PAÍS']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
crear_insertar_tabla("clima", "admin", query_tiburones)

: 